# Probe Designer


## Environment


In [88]:
# basci env
import os
import pandas as pd
import time
import json

# data process of file from ncbi
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import MeltingTemp as mt

# get gene data from ncbi
from Bio import Entrez

# blast and xml file process
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML


# dir
os.chdir("/home/akikawa/repos/probe_designer/dataset/2023.10.24_ChunhongZheng_pancreatic_cancer copy")
current_time = time.localtime()
formatted_time = time.strftime("%Y%m%d_%H%M%S", current_time)
tmp = "./results/" + formatted_time + "/tmp/"
output = "./results/" + formatted_time + "/"
# tmp = './results/' + '20231026_203549' + '/tmp/'
# output = './results/' + '20231026_203549' + '/'
pre_binding_dir = tmp + "pre_binding/"
try:
    os.makedirs(tmp)
except:
    pass

# basic variables
gene_name_list_tosearch = "gene_name_list_tosearch.txt"
pre_binding_file_suffix = "_pre_binding.fasta"
total_pre_binding_file_name = "_total_pre_binding.fasta"

# tmp file
gene_name_list_file = "1_gene_name_list.txt"
gene_id_name_file = "2_id_list.txt"
gene_seq_in_file = "3_gene_seq_in_file.gb"
pre_binding_num_file = "4_pre_binding_num.json"
blast_results_file = "5_blast_results.xml"

## Get genbank file of each gene from ncbi dataset

https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch


In [89]:
# Get gene id and other information from ncbi dataset(api)
## Generate gene_search_list from gene_name_list
organism_of_interest = "Homo sapiens"
n_type_of_interest = "mRNA"
with open(tmp + gene_name_list_file) as f:
    gene_name_list = f.read().splitlines()
gene_search_list = [
    ", ".join([name, organism_of_interest, n_type_of_interest])
    for name in gene_name_list
]

In [161]:
## Get gene id list using Entrez.esearch
id_list = []
for gene_search in gene_search_list:
    Entrez.email = "1418767067@qq.com"
    handle = Entrez.esearch(db="nuccore", term=gene_search)
    record = Entrez.read(handle)
    handle.close()
    id_list += record["IdList"][:1]  # set number of search results to read
with open(tmp + gene_id_name_file, "w") as f:
    f.write("\n".join(id_list))

In [90]:
## Read id_list from existing file
with open(tmp + gene_id_name_file, "r") as f:
    id_list = f.read().split("\n")

In [91]:
# Get the genbank file of each gene by search for id list
fetch_per_round = 3
round = -(-len(id_list) // fetch_per_round)

# initialization of gb file
with open(tmp + gene_seq_in_file, "w") as f:
    f.write("")

for i in range(round):
    id_list_per_round = id_list[i * fetch_per_round : (i + 1) * fetch_per_round]
    Entrez.email = "1418767067@qq.com"
    handle = Entrez.efetch(
        db="nuccore",
        strand=1,  # plus if strand=1
        id=id_list_per_round,
        rettype="gbwithparts",
        retmode="text",
    )
    seq_record = handle.read()
    handle.close()
    print(i + 1, "{:.1f} %".format((i + 1) / round * 100))
    with open(tmp + gene_seq_in_file, "a") as f:
        f.write(seq_record)

1 100.0 %


## Binding site Searcher


In [101]:
import random
from tqdm import tqdm


def select_random_non_overlapping_substrings(input_string, length, num_substrings):
    if length > len(input_string) or num_substrings * length > len(input_string):
        # if length > len(input_string):
        raise ValueError("Invalid input parameters.")

    available_positions = list(range(len(input_string) - length + 1))
    random.shuffle(available_positions)

    substrings = []
    positions = set()  # To keep track of selected positions

    for _ in range(num_substrings):
        if not available_positions:
            break  # Stop if there are no more non-overlapping positions

        # Try to find a non-overlapping position
        selected_position = available_positions.pop()
        while selected_position in positions:
            if not available_positions:
                break
            selected_position = available_positions.pop()

        if selected_position not in positions:
            positions.add(selected_position)
            end = selected_position + length
            selected_substring = input_string[selected_position:end]
            substrings.append(selected_substring)

    return substrings


def find_max_min_difference_fixed_length_subsequence(
    arr,
    length,
    min_gap,
    better_gap=80,
    gene="",
):
    arr.sort()  # 对输入列表进行排序

    def is_valid(min_difference, length, min_gap):
        count = 1
        current_min = arr[0]

        for i in range(1, len(arr)):
            if arr[i] - current_min >= min_difference:
                count += 1
                current_min = arr[i]

        return count >= length and min_difference > min_gap

    left, right = 0, arr[-1] - arr[0]
    result = []
    while left <= right:
        mid = (left + right) // 2
        if is_valid(mid, length, min_gap):
            result = [arr[0]]
            current_min = arr[0]
            for i in range(1, len(arr)):
                if arr[i] - current_min >= mid:
                    result.append(arr[i])
                    current_min = arr[i]
            left = mid + 1
        else:
            right = mid - 1

    if result == []:
        print(f"Gene {gene}: \tNot enough pos for {length} binding sites.")
        result = arr

    if mid < better_gap:
        print(f"Gene {gene}: \tcondition too harsh, loose to get better results")
        print(result)

    return result


def step_by_step(
    sequence,
    BDS_len,
    BDS_num,
    min_gap,
    better_gap,
    gene="",
    G_min=0.3,
    G_max=0.7,
    G_consecutive=5,
    Tm_low=50,
    Tm_high=65,
    pin_gap=0.1,
):
    seq_gap = int(len(sequence) * pin_gap)
    sequence = sequence[seq_gap : len(sequence) - seq_gap]
    position = [_ for _ in range(len(sequence) - BDS_len)]
    pos_of_True = []
    Tm_l_list = [0] * len(position)
    Tm_r_list = [0] * len(position)
    for pos in tqdm(position, desc=f"position_searching_{gene}"):
        bds = sequence[pos : pos + BDS_len]
        # check G 40%-70%, non consective 5 base
        if "G" * G_consecutive in bds:
            continue
        G_per = bds.count("G") / len(bds)
        if G_per < G_min or G_per > G_max:
            continue
        # check Tm
        Tm_l = mt.Tm_NN(bds[: BDS_len // 2], nn_table=mt.R_DNA_NN1)
        Tm_r = mt.Tm_NN(bds[BDS_len // 2 :], nn_table=mt.R_DNA_NN1)
        if Tm_l > Tm_high or Tm_l < Tm_low or Tm_r > Tm_high or Tm_r < Tm_low:
            continue
        pos_of_True.append(pos)
        Tm_l_list[pos] = Tm_l
        Tm_r_list[pos] = Tm_r

    best_pos = find_max_min_difference_fixed_length_subsequence(
        pos_of_True,
        BDS_num,
        min_gap=min_gap,
        better_gap=better_gap,
        gene=gene,
    )
    Tm_l = [Tm_l_list[_] for _ in best_pos]
    Tm_r = [Tm_r_list[_] for _ in best_pos]
    seq_out = [sequence[_ : _ + BDS_len] for _ in best_pos]

    return (
        Tm_l,
        Tm_r,
        seq_out,
        [_ + seq_gap for _ in best_pos],
    )

In [93]:
# Initiation of array
binding_site_FOIs = [
    "accession",
    "gene_name",
    "mol_type",
    "organism",
    "pos_on_seq",
    "binding",
    "Tm_l",
    "Tm_r",
    "wanted",
]
align_FOIs = ["align_num", "align_accession", "align_descrip", "plus/minus"]
FOI = pd.DataFrame(columns=binding_site_FOIs + align_FOIs)

# Search binding sites on mRNA sequence
file_in = tmp + gene_seq_in_file
file_out_dir = pre_binding_dir
try:
    os.mkdir(file_out_dir)
except:
    pass

pre_binding_num = {}

# initialization of file
with open(file_out_dir + total_pre_binding_file_name, "w") as handle:
    handle.write("")

for gene_seq_gb in SeqIO.parse(tmp + gene_seq_in_file, "genbank"):
    # get information of gene
    id = gene_seq_gb.id  # get seq id

    # get gene_name
    if gene_seq_gb.features:
        for feature in gene_seq_gb.features:
            if feature.type == "CDS":
                gene_name = feature.qualifiers.get("gene", ["NAN"])[0]

    # get molecule_type
    mol_type = gene_seq_gb.annotations["molecule_type"]

    # get organism
    organism = gene_seq_gb.annotations["organism"]

    # get minus seq
    translib = {"A": "T", "T": "A", "C": "G", "G": "C"}
    try:
        seq_minus = [translib[i] for i in str(gene_seq_gb.seq)]
        seq = "".join(list(reversed(seq_minus)))
    except:
        seq = str(gene_seq_gb.seq)

    record_list = []
    file_out = file_out_dir + gene_name + pre_binding_file_suffix

    Tm_l, Tm_r, selected_substrings, pos_on_seq = step_by_step(
        seq,
        BDS_len=40,
        BDS_num=50,
        min_gap=1,
        better_gap=40,
        gene=gene_name,
        G_min=0.25,
        G_max=0.7,
        G_consecutive=5,
        Tm_low=50,
        Tm_high=65,
    )

    for i, pre_binding_tmp in enumerate(selected_substrings):
        record_list.append(
            SeqRecord(
                Seq(pre_binding_tmp),
                id="pre_binding" + str(i),
                description="|".join([id, gene_name, organism, mol_type]),
            )
        )

    # add information about binding sites to FOI
    add = pd.DataFrame(
        {
            "accession": [id] * len(selected_substrings),
            "gene_name": [gene_name] * len(selected_substrings),
            "mol_type": [mol_type] * len(selected_substrings),
            "organism": [organism] * len(selected_substrings),
            "binding": selected_substrings,
            "Tm_l": Tm_l,
            "Tm_r": Tm_r,
            "pos_on_seq": pos_on_seq,
        }
    )
    FOI = pd.concat([FOI, add], ignore_index=True)

    # pos += pre_binding_num_tmp

    # write pre_binding to files
    with open(file_out, "w") as f:
        for new_record in record_list:
            SeqIO.write(new_record, f, "fasta")
    with open(file_out_dir + total_pre_binding_file_name, "a") as handle:
        for new_record in record_list:
            SeqIO.write(new_record, handle, "fasta")

    # record the num of pre_binding for each gene
    # pre_binding_num[f"{id}_{gene_name}"] = pre_binding_num_tmp
    pre_binding_num[f"{id}_{gene_name}"] = len(selected_substrings)

with open(tmp + pre_binding_num_file, "w") as f:
    json.dump(pre_binding_num, f)

position_searching_CTRB1: 100%|██████████| 656/656 [00:00<00:00, 11751.96it/s]

condition for gene CTRB1 too harsh, loose to get better results
[2, 12, 21, 33, 42, 51, 67, 119, 129, 138, 151, 160, 173, 185, 194, 214, 234, 243, 252, 265, 274, 285, 294, 303, 313, 323, 333, 342, 353, 362, 371, 422, 433, 442, 451, 460, 484, 499, 508, 519, 528, 537, 546, 555, 566, 575, 586, 595, 604, 623, 639, 648]


## Blast and extract blast results

NCBIXML: https://homolog.us/Biopython/Bio.Blast.NCBIXML.html#read/0

BlastRecord: https://biopython.org/docs/1.75/api/Bio.Blast.Record.html

XMLReader: https://codebeautify.org/xmlviewer#


In [ ]:
with open(file_out_dir + total_pre_binding_file_name, "r") as f:
    fasta_string = f.read()
txid = [2697049]  # organism

# Submit BLAST search and get handle object
handle = NCBIWWW.qblast(
    program="blastn",
    megablast="yes",
    database="refseq_rna",
    sequence=fasta_string,
    url_base="https://blast.ncbi.nlm.nih.gov/Blast.cgi",
    format_object="Alignment",
    format_type="Xml",
)

# read handle object and save to a file
with open(tmp + blast_results_file, "w") as f:
    f.write(handle.read())

In [94]:
# Extract interested information from blast_results
align_num = []

# read the id/plus-minus part/align_num
with open(tmp + blast_results_file, "r") as blast_output:
    blast_records = NCBIXML.parse(blast_output)
    loca = 0
    for blast_record in blast_records:
        align_accession = []
        align_descrip_list = []
        # get align num of each binding site
        length = len(blast_record.alignments)
        align_num.append(length)
        for i in range(length):
            descrip = blast_record.descriptions[i].title.split("|")
            # get accession and descrip of each align seq
            align_accession.append(descrip[3])
            align_descrip_list.append(descrip[-1])
        FOI.loc[loca, "align_accession"] = "|".join(str(_) for _ in align_accession)

        # add align_descrip to df
        FOI.loc[loca, "align_descrip"] = "|".join(str(_) for _ in align_descrip_list)

        # get plus/minus of each align seq
        p_m = [blast_record.alignments[_].hsps[0].frame[1] for _ in range(length)]

        # add plus/minus to df
        try:
            FOI.loc[loca, "plus/minus"] = ",".join([str(_) for _ in p_m])
        except:
            FOI.loc[loca, "plus/minus"] = "NAN"

        loca += 1

FOI["align_num"] = align_num

## Select wanted binding site


In [95]:
FOI["wanted"] = [True] * len(FOI)

In [96]:
# sieve for the suitable binding site
gene_name_list = [_.upper() for _ in gene_name_list]
gene_name_list_out = [i for i in gene_name_list]
for i in range(len(FOI)):
    # check gene_name
    gene_name = FOI.loc[i, "gene_name"]
    if gene_name.upper() not in gene_name_list:
        FOI.loc[i, "wanted"] = False
    else:
        try:
            gene_name_list_out.remove(gene_name)
        except:
            pass

    # check DNA or mRNA type
    if FOI.loc[i, "wanted"] == True:
        if FOI.loc[i, "mol_type"] != "mRNA":
            FOI.loc[i, "wanted"] = False

    # check gene_organism name
    if FOI.loc[i, "wanted"] == True:
        spe_ori, gene_ori = FOI.loc[i, "organism"], FOI.loc[i, "gene_name"]
        descrip = FOI.loc[i, "align_descrip"].split("|")
        for des in descrip:
            if gene_ori not in des and spe_ori in des:
                FOI.loc[i, "wanted"] = False
                break

    # check plus/minus
    if FOI.loc[i, "wanted"] == True:
        if pd.isnull(FOI.loc[i, "plus/minus"]):
            FOI.loc[i, "wanted"] = False
        else:
            pm_list = FOI.loc[i, "plus/minus"].split(",")
            if "-1" not in pm_list:
                FOI.loc[i, "wanted"] = False

# write the whole information of interest to a excel file in tmp dir
FOI.to_excel(tmp + "probes_sieve.xlsx")

out_tmp = FOI[FOI["wanted"] == True]
output_df = pd.DataFrame()
for gene in out_tmp.gene_name.unique():
    pos_of_True = list(out_tmp[out_tmp.gene_name == gene]["pos_on_seq"])
    best_pos = find_max_min_difference_fixed_length_subsequence(
        pos_of_True,
        length=3,
        min_gap=40,
        better_gap=80,
        gene=gene,
    )
    out_subset = out_tmp[out_tmp.gene_name == gene]
    out_subset = out_subset[out_subset["pos_on_seq"].isin(best_pos)]
    output_df = pd.concat([output_df, out_subset])

# write the output to a xlsx file
output_df.to_excel(output + "probes_wanted.xlsx")

condition for gene CTRB1 too harsh, loose to get better results
[89, 99, 108, 120, 129]
